In [65]:
import random

def csv_to_array(filename):
    file = open(filename, 'r')
    file_text = file.read()
    # Get rid of the first three, which are examples
    split_text = file_text.split('|')[3:]
    i = 0
    tuple_text = []
    while i < len(split_text) - 1:
        tuple_text.append((split_text[i+1], split_text[i], int(i/2)))
        i += 2
    
    return tuple_text
    
def make_training_and_dev_data(all_data):
    random.shuffle(all_data)
    training_ratio = .9
    return all_data[:int(len(all_data) * training_ratio)], all_data[int(len(all_data) * training_ratio):]
#     return all_data[:(len(all_data) * training_ratio)], all_data[:(len(all_data) * training_ratio)] 


official_data = csv_to_array('yelp_data_official_training.csv')

for garbage in official_data:
    if garbage[1].isdigit() is False:
        print(garbage[1])
        break

training_data, dev_data = make_training_and_dev_data(official_data)

In [66]:
def reviews(data):
    reviews = []
    for i in range(len(training_data)):
        reviews.append(training_data[i][0])
    return reviews

def labels(data):
    reviews = []
    for i in range(len(training_data)):
        reviews.append(int(training_data[i][1]))
    return reviews

reviews_training = reviews(training_data)
label_training = labels(training_data)
reviews_dev = reviews(dev_data)
label_dev = labels(dev_data)

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(reviews_training)
X_train_counts.shape

(43200, 88894)

In [68]:
count_vect.vocabulary_.get('and')

45721

In [69]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(43200, 88894)

In [70]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(43200, 88894)

In [71]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, label_training)

In [72]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [73]:
text_clf = text_clf.fit(reviews_training, label_training)

In [74]:
import numpy as np
docs_dev = reviews_dev
predicted = text_clf.predict(docs_dev)
np.mean(predicted == label_dev)

0.74122685185185189

In [75]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)),])
_ = text_clf.fit(reviews_training, label_training)
predicted = text_clf.predict(docs_dev)
np.mean(predicted == label_dev) 

0.85335648148148147

In [77]:
from sklearn import metrics
print(metrics.classification_report(label_training, predicted))

             precision    recall  f1-score   support

          1       0.85      0.95      0.90     13572
          2       0.82      0.87      0.85      7161
          3       0.84      0.95      0.89     10234
          4       0.93      0.68      0.78      7035
          5       0.84      0.75      0.79      3338
          6       0.97      0.41      0.58      1860

avg / total       0.86      0.85      0.85     43200



In [78]:
metrics.confusion_matrix(label_training, predicted)

array([[12882,   123,   494,    71,     2,     0],
       [  518,  6241,   189,   201,     7,     5],
       [  384,    87,  9695,    62,     6,     0],
       [  801,  1053,   388,  4777,     9,     7],
       [   76,    21,   720,     5,  2501,    15],
       [  487,    69,    36,    45,   454,   769]])

In [81]:
b = zip(reviews_training, predicted)
print (b)

In [8]:
docs_new = ['God is love', 'OpenGL on the GPU is fast','dr.']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print(doc, category)

God is love 1
OpenGL on the GPU is fast 1
dr. 3


In [118]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [119]:
bcv =['question rage pizza','further and total']
bcv_label = [1,1]
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])